In [31]:
import pandas as pd

updated_csv = pd.read_csv('data/updated_out_tempo5.csv', dtype={'week': str})
updated_csv = updated_csv.dropna(subset=['rain_max'])

updated_csv.to_csv('updated_out_tempo_drop.dengue_cases_data', index=False)

In [32]:
# Group by ESTACAO and week, then calculate max and mean precipitation

cleaned_csv = pd.read_csv('data/updated_out_tempo_drop.csv', dtype={'week': str})

weekly_stats = cleaned_csv.groupby(['ESTACAO', 'week'])['rain_max'].agg(
    precipitation_max='max',
    precipitation_avg='mean'
).reset_index()

# Select only the required columns, including week
result = weekly_stats[['ESTACAO', 'week', 'precipitation_max', 'precipitation_avg']]

# Save the result to a new CSV file
result.to_csv('final_imputed_weekly_precipitation_drop.dengue_cases_data', index=False)

Data imputation started. the original file is rain_data.csv


In [8]:
import pandas as pd

# Load the original CSV file
original_csv = pd.read_csv('data/rain_data.csv', dtype={'week': str})

# missing values
missing_values = original_csv.isnull().sum()
print(missing_values)

ESTACAO        0
DATA           0
rain_max    1269
dtype: int64


In [11]:
import pandas as pd
import numpy as np
from geopy.distance import geodesic

# Load the rain data and stations data
rain_data = pd.read_csv('data/rain_data.csv')  # Replace with actual file name
stations = pd.read_csv('data/stations_RIO.csv')  # Replace with actual file name

# Function to find the nearest station based on geodesic distance
def find_nearest_station(rain_row, stations_df):
    target_coords = (rain_row['lat'], rain_row['lon'])
    stations_df['distance'] = stations_df.apply(
        lambda row: geodesic(target_coords, (row['lat'], row['lon'])).km, axis=1
    )
    nearest_station = stations_df.loc[stations_df['distance'].idxmin()]
    return nearest_station

# Function to fill missing rain data
def fill_missing_rain_data(rain_data, stations_df):
    # Add lat/lon info to rain_data by matching 'ESTACAO' with 'id_station' from stations_df
    rain_data = rain_data.merge(stations_df[['id_station', 'lat', 'lon']], left_on='ESTACAO', right_on='id_station', how='left')

    # Loop through missing values in rain_max and fill them
    for index, row in rain_data[rain_data['rain_max'].isna()].iterrows():
        # Find the nearest station for the missing data
        nearest_station = find_nearest_station(row, stations_df)

        # Get the rain data for the nearest station (if available) and fill missing values
        station_data = rain_data[rain_data['ESTACAO'] == nearest_station['id_station']]

        if not station_data.empty:
            avg_precipitation = station_data['rain_max'].mean()  # Mean of the nearest station data
            rain_data.at[index, 'rain_max'] = avg_precipitation

    return rain_data

# Fill missing values with the nearest station data
filled_rain_data = fill_missing_rain_data(rain_data, stations)

# Save the updated data to the same file
filled_rain_data.to_csv('filled_rain_data.dengue_cases_data', index=False)

print("Missing rain data filled successfully.")


Missing rain data filled successfully.


In [29]:
import pandas as pd
import numpy as np
from geopy.distance import geodesic

# Load the rain data and stations data
rain_data = pd.read_csv('data/filled_rain_data.csv')  # Replace with actual file name
stations = pd.read_csv('data/stations_RIO.csv')  # Replace with actual file name

# Merge rain_data with stations to get lat/lon for each ESTACAO
rain_data = rain_data.merge(stations[['id_station', 'lat', 'lon']], left_on='ESTACAO', right_on='id_station', how='left')

# Function to find the k nearest stations based on geodesic distance
def find_k_nearest_stations(rain_row, stations_df, k=3):
    target_coords = (rain_row['lat'], rain_row['lon'])
    stations_df['distance'] = stations_df.apply(
        lambda row: geodesic(target_coords, (row['lat'], row['lon'])).km, axis=1
    )
    # Get the k nearest stations
    nearest_stations = stations_df.nsmallest(k, 'distance')
    return nearest_stations

# Function to fill missing rain data using k-nearest neighbors
def fill_missing_rain_data(rain_data, stations_df, k=3):
    # Loop through missing values in rain_max and fill them
    for index, row in rain_data[rain_data['rain_max'].isna()].iterrows():
        # Find the k nearest stations for the missing data
        nearest_stations = find_k_nearest_stations(row, stations_df, k)

        # Get the rain data for the nearest stations and calculate the average
        # Filter the rain_data for the nearest stations and get their 'rain_max' values
        nearest_rain_data = rain_data[rain_data['ESTACAO'].isin(nearest_stations['id_station'])]

        # Calculate the mean of the 'rain_max' from the nearest stations
        avg_precipitation = nearest_rain_data['rain_max'].mean()  # Mean of the k nearest stations' data
        rain_data.at[index, 'rain_max'] = avg_precipitation

    return rain_data

# Fill missing values with the nearest station data (k=3)
filled_rain_data = fill_missing_rain_data(rain_data, stations, k=3)

# Save the updated data to the same file
filled_rain_data.to_csv('filled_rain_data.dengue_cases_data', index=False)

print("Missing rain data filled successfully.")


Missing rain data filled successfully.


In [27]:
import pandas as pd
import numpy as np
from geopy.distance import geodesic

# Load the rain data and stations data
rain_data = pd.read_csv('data/filled_rain_data.csv')  # Replace with actual file name
stations = pd.read_csv('data/stations_RIO.csv')  # Replace with actual file name

# Merge rain_data with stations to get lat/lon for each ESTACAO
rain_data = rain_data.merge(stations[['id_station', 'lat', 'lon']], left_on='ESTACAO', right_on='id_station', how='left')

# Function to find the k nearest stations based on geodesic distance
def find_k_nearest_stations(rain_row, stations_df, k=3):
    target_coords = (rain_row['lat'], rain_row['lon'])
    stations_df['distance'] = stations_df.apply(
        lambda row: geodesic(target_coords, (row['lat'], row['lon'])).km, axis=1
    )
    # Get the k nearest stations
    nearest_stations = stations_df.nsmallest(k, 'distance')
    return nearest_stations

# Function to fill missing rain data using k-nearest neighbors
def fill_missing_rain_data(rain_data, stations_df, k=3):
    # Loop through missing values in rain_max and fill them
    for index, row in rain_data[rain_data['rain_max'].isna()].iterrows():
        # Find the k nearest stations for the missing data
        nearest_stations = find_k_nearest_stations(row, stations_df, k)

        # Filter the rain_data for the nearest stations and the specific date (DATA)
        nearest_rain_data = rain_data[(rain_data['ESTACAO'].isin(nearest_stations['id_station'])) &
                                      (rain_data['DATA'] == row['DATA'])]

        # Calculate the mean of the 'rain_max' from the nearest stations for the specific date
        avg_precipitation = nearest_rain_data['rain_max'].mean()  # Mean of the k nearest stations' data
        rain_data.at[index, 'rain_max'] = avg_precipitation

    return rain_data

# Fill missing values with the nearest station data (k=3)
filled_rain_data = fill_missing_rain_data(rain_data, stations, k=1)

# Save the updated data to the same file
filled_rain_data.to_csv('filled_rain_data.dengue_cases_data', index=False)

print("Missing rain data filled successfully.")


Missing rain data filled successfully.


In [30]:
filled_rain_data = pd.read_csv('data/filled_rain_data.csv')

# missing values
missing_values = filled_rain_data.isnull().sum()
print(missing_values)

# only take data and rain_max columns
filled_rain_data = filled_rain_data[['ESTACAO', 'DATA', 'rain_max']]

# Save the updated data to a new file
filled_rain_data.to_csv('filled_rain_data.dengue_cases_data', index=False)

ESTACAO       0
DATA          0
rain_max      0
id_station    0
lat           0
lon           0
dtype: int64
